In [1]:
import pandas as pd

df = pd.read_csv("crm_opportunities.csv")
df['Signup_Date'] = pd.to_datetime(df['Signup_Date'], errors='coerce')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   User_ID         500 non-null    object        
 1   Signup_Date     500 non-null    datetime64[ns]
 2   Bet_Amount      500 non-null    float64       
 3   Region          500 non-null    object        
 4   Account_Status  500 non-null    object        
 5   Login_Count     500 non-null    int64         
 6   Customer_Type   500 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 27.5+ KB


#### Normallik, Çarpıklık ,Basıklık

##### Shapiro-wilks

In [4]:
from scipy import stats

numeric_columns = df.drop(columns=['User_ID','Signup_Date','Region','Account_Status','Customer_Type'])

for column in numeric_columns.columns:
    data = pd.to_numeric(df[column], errors='coerce').dropna()
    stat, p_value = stats.shapiro(data)
    
    print(f'{column} için Shapiro-Wilk testi:')
    print(f'  Test istatistiği: {stat}')
    print(f'  p-değeri: {p_value}\n')

    if p_value < 0.05:
        print(f'{column} verisi normal dağılmıyor.\n')
    else:
        print(f'{column} verisi normal dağılıyor.\n')

Bet_Amount için Shapiro-Wilk testi:
  Test istatistiği: 0.9516400694847107
  p-değeri: 1.008529718071749e-11

Bet_Amount verisi normal dağılmıyor.

Login_Count için Shapiro-Wilk testi:
  Test istatistiği: 0.9552814960479736
  p-değeri: 3.595929110034035e-11

Login_Count verisi normal dağılmıyor.



##### Çarpıklık ve Basıklık

In [6]:
import pandas as pd

numeric_columns = df.drop(columns=['User_ID','Signup_Date','Region','Account_Status','Customer_Type'])

for column in numeric_columns.columns:
    skewness = numeric_columns[column].skew()  # Çarpıklık
    kurtosis = numeric_columns[column].kurt()  # Basıklık
    
    if skewness > 2:
        skewness_level = 'Şiddetli sağa çarpık'
    elif skewness > 1:
        skewness_level = 'Orta derecede sağa çarpık'
    elif skewness > 0.5:
        skewness_level = 'Hafif sağa çarpık'
    elif skewness > -0.5:
        skewness_level = 'Simetrik'
    elif skewness > -1:
        skewness_level = 'Hafif sola çarpık'
    elif skewness > -2:
        skewness_level = 'Orta derecede sola çarpık'
    else:
        skewness_level = 'Şiddetli sola çarpık'
    
    if kurtosis > 6:
        kurtosis_level = 'Aşırı leptokurtik (çok belirgin uç noktalar)'
    elif kurtosis > 3:
        kurtosis_level = 'Leptokurtik (keskin dağılım)'
    elif kurtosis > 0:
        kurtosis_level = 'Normal dağılıma yakın'
    elif kurtosis > -1:
        kurtosis_level = 'Daha düz, platykurtik'
    else:
        kurtosis_level = 'Aşırı düz'
    

    print(f'{column} için Çarpıklık (Skewness): {skewness} - {skewness_level}')
    print(f'{column} için Basıklık (Kurtosis): {kurtosis} - {kurtosis_level}\n')

Bet_Amount için Çarpıklık (Skewness): 0.03196082895312164 - Simetrik
Bet_Amount için Basıklık (Kurtosis): -1.2291471597711463 - Aşırı düz

Login_Count için Çarpıklık (Skewness): -0.030628799230582506 - Simetrik
Login_Count için Basıklık (Kurtosis): -1.1701370163410936 - Aşırı düz



#### Korelasyon Testleri

##### Spearman Korelasyonu

In [9]:
from scipy.stats import spearmanr

corr, p = spearmanr(df['Bet_Amount'], df['Login_Count'])
print(f"Spearman korelasyonu: {corr:.3f}, p-değeri: {p:.4f}")


Spearman korelasyonu: -0.039, p-değeri: 0.3879


##### Kendall’s Tau

In [11]:
from scipy.stats import kendalltau

corr, p = kendalltau(df['Bet_Amount'], df['Login_Count'])
print(f"Kendall's Tau: {corr:.3f}, p-değeri: {p:.4f}")


Kendall's Tau: -0.024, p-değeri: 0.4298


In [12]:
from scipy.stats import spearmanr, kendalltau
import pandas as pd


spearman_corr, spearman_p = spearmanr(df['Bet_Amount'], df['Login_Count'])
kendall_corr, kendall_p = kendalltau(df['Bet_Amount'], df['Login_Count'])


corr_df = pd.DataFrame({
    'Test': ['Spearman', 'Kendall\'s Tau'],
    'Korelasyon Katsayısı': [spearman_corr, kendall_corr],
    'p-değeri': [spearman_p, kendall_p],
    'Yorum': [
        'Pozitif ilişki, anlamlı' if spearman_p < 0.05 else 'Anlamsız ilişki',
        'Pozitif ilişki, anlamlı' if kendall_p < 0.05 else 'Anlamsız ilişki'
    ]
})

print("\n📊 Korelasyon Sonuçları Tablosu:\n")
print(corr_df.to_string(index=False))



📊 Korelasyon Sonuçları Tablosu:

         Test  Korelasyon Katsayısı  p-değeri           Yorum
     Spearman             -0.038700  0.387858 Anlamsız ilişki
Kendall's Tau             -0.023681  0.429756 Anlamsız ilişki


#### Levene Testi

In [14]:
from scipy.stats import levene

categorical_cols = ['Region', 'Account_Status', 'Customer_Type']
target = 'Bet_Amount'

for col in categorical_cols:
    groups = [group[target].values for name, group in df.groupby(col)]
    stat, p = levene(*groups)
    print(f"Levene testi sonucu: {col} için")
    print(f"  İstatistik: {stat:.4f}, p-değeri: {p:.4f}")
    if p < 0.05:
        print(f"  → {col} gruplarının varyansları eşit değil (anlamlı fark var).")
    else:
        print(f"  → {col} gruplarının varyansları eşit kabul edilebilir.")
    print("-"*50)



Levene testi sonucu: Region için
  İstatistik: 0.8553, p-değeri: 0.5111
  → Region gruplarının varyansları eşit kabul edilebilir.
--------------------------------------------------
Levene testi sonucu: Account_Status için
  İstatistik: 0.8929, p-değeri: 0.4101
  → Account_Status gruplarının varyansları eşit kabul edilebilir.
--------------------------------------------------
Levene testi sonucu: Customer_Type için
  İstatistik: 0.0150, p-değeri: 0.9851
  → Customer_Type gruplarının varyansları eşit kabul edilebilir.
--------------------------------------------------


heterodaksisite (varyansların eşitsizliği) yok.

#### Kruskal-Wallis

In [17]:
from scipy.stats import mannwhitneyu, kruskal

categorical_cols = ['Region', 'Account_Status', 'Customer_Type']
target = 'Bet_Amount'

for col in categorical_cols:
    groups = [group[target].values for name, group in df.groupby(col)]
    n_groups = len(groups)
    print(f"=== {col} için test ===")
    
    if n_groups == 2:
        stat, p = mannwhitneyu(*groups)
        test_name = "Mann-Whitney U testi"
    elif n_groups > 2:
        stat, p = kruskal(*groups)
        test_name = "Kruskal-Wallis testi"
    else:
        print(f"{col} için yeterli grup yok, atlandı.")
        continue
    
    print(f"{test_name} istatistiği: {stat:.4f}, p-değeri: {p:.4f}")
    if p < 0.05:
        print(f"→ {col} gruplar arasında anlamlı fark var.")
    else:
        print(f"→ {col} gruplar arasında anlamlı fark yok.")
    print("-"*40)


=== Region için test ===
Kruskal-Wallis testi istatistiği: 1.6079, p-değeri: 0.9003
→ Region gruplar arasında anlamlı fark yok.
----------------------------------------
=== Account_Status için test ===
Kruskal-Wallis testi istatistiği: 1.8922, p-değeri: 0.3882
→ Account_Status gruplar arasında anlamlı fark yok.
----------------------------------------
=== Customer_Type için test ===
Kruskal-Wallis testi istatistiği: 3.4932, p-değeri: 0.1744
→ Customer_Type gruplar arasında anlamlı fark yok.
----------------------------------------


Bet_Amount değerleri, bu üç kategorik değişkenin grupları arasında istatistiksel olarak anlamlı farklılık göstermiyor.

Yani, örneğin Region gruplarının bahis miktarları medyan olarak birbirinden anlamlı şekilde farklı değil.

Benzer şekilde, Account_Status ve Customer_Type gruplarının da Bet_Amount değerlerinde anlamlı bir fark yok.

Bu sonuçlar, gruplar arası Bet_Amount farkının düşük olduğunu veya örneklem büyüklüğünün bu farkı yakalamaya yetmediğini gösteriyor olabilir.

Eğer amaç gruplar arası fark bulmaksa, farklı değişkenler veya yeni özellikler oluşturup tekrar deneyebiliriz.

Ya da müşteri davranışlarını başka yöntemlerle (segmentasyon, zaman serileri, regresyon vs.) inceleyebiliriz.

#### Mutual Regression

In [21]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

drop_cols = ['User_ID', 'Signup_Date', 'Bet_Amount']


X = df.drop(columns=drop_cols)


categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()


encoder = OneHotEncoder(sparse=False)
X_cat_encoded = encoder.fit_transform(X[categorical_cols])


X_num = X[numerical_cols].values


X_all = np.hstack((X_num, X_cat_encoded))


y = df['Bet_Amount'].values


discrete_features = [False]*len(numerical_cols) + [True]*X_cat_encoded.shape[1]


mi = mutual_info_regression(X_all, y, discrete_features=discrete_features, random_state=42)

feature_names = numerical_cols + list(encoder.get_feature_names_out(categorical_cols))


mi_results = pd.DataFrame({'Feature': feature_names, 'Mutual_Info': mi})
mi_results = mi_results.sort_values(by='Mutual_Info', ascending=False).reset_index(drop=True)

print(mi_results)


                      Feature  Mutual_Info
0    Customer_Type_High-value     0.018596
1            Region_Australia     0.006581
2        Region_South America     0.005060
3   Account_Status_Fraudulent     0.001475
4                 Login_Count     0.000000
5               Region_Africa     0.000000
6                 Region_Asia     0.000000
7               Region_Europe     0.000000
8        Region_North America     0.000000
9       Account_Status_Active     0.000000
10   Account_Status_Suspended     0.000000
11          Customer_Type_New     0.000000
12      Customer_Type_Regular     0.000000


#### Ki-kare Testi ve Cramér’s V Hesaplama

In [23]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

categorical_cols = ['Region', 'Account_Status', 'Customer_Type']

for i in range(len(categorical_cols)):
    for j in range(i+1, len(categorical_cols)):
        var1 = categorical_cols[i]
        var2 = categorical_cols[j]
        
        contingency_table = pd.crosstab(df[var1], df[var2])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        cv = cramers_v(contingency_table)
        
        print(f"{var1} - {var2} arasındaki ilişki:")
        print(f"  Ki-kare p-değeri: {p:.4f}")
        print(f"  Cramér's V: {cv:.4f}")
        if p < 0.05:
            print("  → Anlamlı ilişki var.")
        else:
            print("  → Anlamlı ilişki yok.")
        print("-"*40)


Region - Account_Status arasındaki ilişki:
  Ki-kare p-değeri: 0.3289
  Cramér's V: 0.0369
  → Anlamlı ilişki yok.
----------------------------------------
Region - Customer_Type arasındaki ilişki:
  Ki-kare p-değeri: 0.7995
  Cramér's V: 0.0000
  → Anlamlı ilişki yok.
----------------------------------------
Account_Status - Customer_Type arasındaki ilişki:
  Ki-kare p-değeri: 0.9104
  Cramér's V: 0.0000
  → Anlamlı ilişki yok.
----------------------------------------


Bu üç kategorik değişken arasında anlamlı ve güçlü bir ilişki yok. Yani örneğin, Region’un Account_Status ya da Customer_Type üzerinde anlamlı bir etkisi görünmüyor.

In [25]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor


numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()


X = df[numerical_cols]


vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]


def vif_comment(vif):
    if vif < 5:
        return "Multicollinearity yok veya çok düşük"
    elif 5 <= vif < 10:
        return "Orta düzeyde multicollinearity, dikkat edilmeli"
    else:
        return "Yüksek multicollinearity, problem olabilir!"

vif_data["Yorum"] = vif_data["VIF"].apply(vif_comment)


print(vif_data)


       Feature       VIF                                 Yorum
0   Bet_Amount  2.246215  Multicollinearity yok veya çok düşük
1  Login_Count  2.246215  Multicollinearity yok veya çok düşük


| Test Türü                   | Değişkenler / Grup           | İstatistik / Değer            | p-değeri     | Yorum / Anlamlılık                  |
|----------------------------|-----------------------------|-------------------------------|--------------|------------------------------------|
| Shapiro-Wilk (Normallik)   | Bet_Amount                  | 0.9516                        | 1.01e-11     | Normal dağılım yok                 |
| Shapiro-Wilk (Normallik)   | Login_Count                 | 0.9553                        | 3.59e-11     | Normal dağılım yok                 |
| Çarpıklık (Skewness)       | Bet_Amount                  | 0.032                         | -            | Simetrik                          |
| Çarpıklık (Skewness)       | Login_Count                 | -0.031                        | -            | Simetrik                          |
| Basıklık (Kurtosis)        | Bet_Amount                  | -1.229                        | -            | Aşırı düz (platykurtik)           |
| Basıklık (Kurtosis)        | Login_Count                 | -1.170                        | -            | Aşırı düz (platykurtik)           |
| Spearman Korelasyon        | Bet_Amount - Login_Count    | -0.0387                      | 0.388        | Anlamsız ilişki                   |
| Kendall's Tau              | Bet_Amount - Login_Count    | -0.0237                      | 0.430        | Anlamsız ilişki                   |
| Levene Testi (Varyans Eşitliği) | Region                   | 0.8553                       | 0.511        | Varyanslar eşit kabul edilebilir  |
| Levene Testi (Varyans Eşitliği) | Account_Status            | 0.8929                       | 0.410        | Varyanslar eşit kabul edilebilir  |
| Levene Testi (Varyans Eşitliği) | Customer_Type             | 0.0150                       | 0.985        | Varyanslar eşit kabul edilebilir  |
| Kruskal-Wallis Testi       | Bet_Amount ~ Region         | 1.608                        | 0.900        | Gruplar arası anlamlı fark yok     |
| Kruskal-Wallis Testi       | Bet_Amount ~ Account_Status | 1.892                        | 0.388        | Gruplar arası anlamlı fark yok     |
| Kruskal-Wallis Testi       | Bet_Amount ~ Customer_Type  | 3.493                        | 0.174        | Gruplar arası anlamlı fark yok     |
| Mutual Information         | Customer_Type_High-value    | 0.0186                       | -            | En yüksek bilgi katkısı            |
| Mutual Information         | Region_Australia            | 0.0066                       | -            | Düşük bilgi katkısı               |
| Mutual Information         | Diğer kategorik değişkenler | ≤ 0.005                      | -            | İlgisiz veya çok düşük katkı       |
| Ki-Kare ve Cramér's V      | Region - Account_Status     | p=0.329, Cramér's V=0.037    | -            | Anlamlı ilişki yok                |
| Ki-Kare ve Cramér's V      | Region - Customer_Type      | p=0.800, Cramér's V=0.000    | -            | Anlamlı ilişki yok                |
| Ki-Kare ve Cramér's V      | Account_Status - Customer_Type | p=0.910, Cramér's V=0.000  | -            | Anlamlı ilişki yok                |
| VIF (Çoklu Bağlantı)       | Bet_Amount                  | 2.25                         | -            | Multicollinearity yok veya çok düşük |
| VIF (Çoklu Bağlantı)       | Login_Count                 | 2.25                         | -            | Multicollinearity yok veya çok düşük |


# Veri Analizi Özeti ve Model Önerileri

## 1. Veri Dağılımı
- Bet_Amount ve Login_Count **normal dağılmıyor**.
- Her iki değişken **simetrik** ve **aşırı düz (platykurtik)** dağılıma sahip.

## 2. Değişkenler Arası İlişki
- Bet_Amount ile Login_Count arasında anlamlı bir korelasyon yok.
- Kategorik değişkenler (Region, Account_Status, Customer_Type) ile Bet_Amount arasında anlamlı fark bulunmadı.

## 3. Varyans Homojenliği
- Kategorik gruplar arasında varyans eşitliği sağlanıyor (Levene testi).

## 4. Kategorik Değişkenlerin Bilgi Katkısı
- Mutual Information değerleri genel olarak düşük.
- Customer_Type_High-value değişkeni en yüksek ama düşük bilgi katkısına sahip.

## 5. Kategorik Değişkenler Arasındaki İlişki
- Ki-kare ve Cramér's V testleri anlamlı ilişki göstermedi.

## 6. Çoklu Bağlantı (Multicollinearity)
- Bet_Amount ve Login_Count arasında çoklu bağlantı yok (VIF ≈ 2.25).

---

# Model Önerileri

- **Parametrik olmayan modeller** veya **veri dönüşümleri** tercih edilmeli (log, Box-Cox).
- **Tree-based modeller** (Random Forest, XGBoost vb.) normallik gerektirmediğinden uygun.
- Kategorik değişkenler tek başına güçlü tahminçi değil; **feature engineering** yapılabilir.
- Segmentasyon veya farklı davranışsal analizler için yeni kriterler ve veri eklenmeli.

---

# İş Zekası İçin
- Mevcut kategorik gruplar Bet_Amount’da anlamlı farklılık yaratmıyor.
- Kullanıcı davranışları bağımsız değişkenlerle karmaşık, detaylı analiz ve yeni veri gerekebilir.
- Modelleme öncesi veri dönüşümü ve uygun model seçimi kritik.

